In [1]:
# imports
import xgboost as xgb
import pandas as pd
import yaml
from python_scripts.model_performance import generate_df_summary

# import model
model = xgb.XGBClassifier()
model.load_model('saved_models/final_model/final_model.json')

# import test data
X_test = pd.read_parquet('data/X_test.parquet')
y_test = pd.read_csv('data/y_test.csv')


In [2]:
# predict on test set using optimised threshold
with open('model_settings.yaml', 'r') as file:
    model_settings = yaml.safe_load(file)
best_threshold = model_settings['best_threshold']

y_proba = model.predict_proba(X_test)[:, 1]
y_pred = (y_proba >= best_threshold).astype(int)

In [19]:
generate_df_summary(model, y_test, y_pred, 'final_model_test', threshold=best_threshold)


Precision: 0.28
Recall: 0.27
F1-Score: 0.28

Confusion Matrix:

               Pred Class 0  Pred Class 1
True Class 0         94434           981
True Class 1          1041           387
